In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
from statistics import mean

import requests

Data for evaluating careers of NFL WR's:

NFL stats (2012-2022)

In [3]:
le_data = pd.read_csv('Data/yearly_stats.csv', dtype={'height': str}, parse_dates= False)
#issue as orignally I thought pandas was interpreting height as a date, but the csv is reporting heights as dates

#filter for WR postion since that is all we care about

WR_data = le_data[le_data['position'] == 'WR']

#lets only return relevant columns (recieving production)

WR_data = WR_data[['id', 'name', 'position', 'team', 'season', 'season_type',
         'receptions', 'targets', 'receiving_yards',
       'receiving_tds','receiving_yards_after_catch',
       'receiving_first_downs','target_share',
       'air_yards_share', 'fantasy_points', 'fantasy_points_ppr',
         'games', 'offense_snaps', 'teams_offense_snaps',
       'ypr','rec_td_percentage','rec_ypg',
         'round', 'overall', 'ht', 'wt', 'forty', 'vertical']]

#had issue with DK Metcalf as this dataset has him at DK wheras madden ratings name him D.K.
WR_data['name'] = WR_data['name'].replace('DK Metcalf', 'D.K. Metcalf')
#DJ Chark too, madden name him D.J.
WR_data['name'] = WR_data['name'].replace('D.J. Chark', 'D.J. Chark Jr')

Now have to merge the Stats Data with Madden ratings for each year.

Note: I originally used ratings from the Madden game associated with that season. For example, during the 2022 NFL season, Madden 23 was the latest
edition of the game. However, Madden 23 would have the rating for the player prior to the start of the 2022 season, whereas I am looking for a rating
that encapsualtes a player's performance for that season. Thus, for the 2022 NFL season, Madden 24 has ratings that are based off of 2022 NFL performance. In that sense, Madden ratings are 2 values ahead.

In [189]:
#madden 21 ratings for WRs (2019 NFL season)
madden21 = pd.read_csv('Data/maddenratings/madden21_ratings.csv')
madden21 = madden21[['Full Name', 'Overall Rating', 'Position']]
madden21WR = madden21[madden21['Position'] == 'WR']
madden21WR = madden21WR[['Full Name', 'Overall Rating']]

#add the season
madden21WR['season'] = 2019
madden21WR['season'] = madden21WR['season'].astype(int)

#renaming the 'Full Name' column to 'name' so it merge with WR_data easier
madden21WR.rename(columns = {'Full Name': 'name'}, inplace= True)

#see note on marvin jones issue
madden21WR['name'] = madden21WR['name'].replace('Marvin Jones Jr', 'Marvin Jones')

In [190]:
#merging
WR_data2 = pd.merge(WR_data, madden21WR, on= ['name', 'season'], how='left')

In [229]:
#madden 23 ratings for WRs (2021 NFL season)
madden23 = pd.read_csv('Data/maddenratings/Madden23Ratings.csv')
madden23 = madden23[['fullNameForSearch','position','overall_rating']]
madden23 = madden23[madden23['position'] == 'WR']
madden23 = madden23[['fullNameForSearch','overall_rating']]

#add the season
madden23['season'] = 2021
madden23['season'] = madden23['season'].astype(int) #not sure if this is necessary but wtv

#renaming columns for merge

madden23.rename(columns = {'fullNameForSearch':'name', 'overall_rating':'Overall Rating'}, inplace=True)

#marvin jones issue
madden23['name'] = madden23['name'].replace('Marvin Jones Jr', 'Marvin Jones')

In [230]:
#merging
WR_data3 = pd.merge(WR_data2, madden23, on = ['name','season'], how= 'left')

WR_data3['Overall Rating'] = np.where(WR_data3['season'] == 2019, 
                                      WR_data3['Overall Rating_x'], 
                                      np.where(WR_data3['season'] == 2021, 
                                               WR_data3['Overall Rating_y'], 
                                               np.nan))

WR_data3.drop(columns=['Overall Rating_x', 'Overall Rating_y'], inplace=True)

In [250]:
#madden 24 ratings for WRs (2022 NFL season)
madden24 = pd.read_csv('Data/maddenratings/madden24.csv')
madden24 = madden24[['Full Name', 'Position', 'Overall Rating']]
madden24 = madden24[madden24['Position'] == 'WR']
madden24 = madden24[['Full Name', 'Overall Rating']]

#add season
madden24['season'] = 2022
madden24['season'] = madden24['season'].astype(int)

#rename columns
madden24.rename(columns = {'Full Name':'name'}, inplace=True)

#marv
madden24['name'] = madden24['name'].replace('Marvin Jones Jr', 'Marvin Jones')

In [251]:
#merging
WR_data4 = pd.merge(WR_data3, madden24, on = ['name','season'], how = 'left',suffixes=('', '_2023'))

WR_data4['Overall Rating'] = np.where(WR_data4['season'] == 2022, 
                                      WR_data4['Overall Rating_2023'], 
                                      WR_data4['Overall Rating'])

WR_data4.drop(columns=['Overall Rating_2023'], inplace=True)



In [252]:
#madden 22 ratings for WRs (2020 NFL season)
madden22 = pd.read_excel('Data/maddenratings/datatime.xlsx', sheet_name = 'All Players', skiprows = 1)
madden22 = madden22[['First Name', 'Last Name', 'Position', 'Overall']]
madden22 = madden22[madden22['Position'] == 'WR']
madden22 = madden22[['First Name', 'Last Name', 'Overall']]

#create new name columm
madden22['name'] = madden22['First Name'] + ' ' + madden22['Last Name']

# add season
madden22['season'] = 2020
madden22['season'] = madden22['season'].astype(int)

#drop first name last name
madden22 = madden22.drop(columns = ['First Name', 'Last Name'])

#rename columns
madden22.rename(columns = {'Overall': 'Overall Rating'}, inplace= True)

#marvin jones returns
madden22['name'] = madden22['name'].replace('Marvin Jones Jr', 'Marvin Jones')

In [253]:
#merging
WR_data_final = pd.merge(WR_data4, madden22, on= ['name','season'], how = 'left',suffixes=('', '_2022'))


WR_data_final['Overall Rating'] = np.where(WR_data_final['season'] == 2020,
                                           WR_data_final['Overall Rating_2022'],
                                           WR_data_final['Overall Rating'])

WR_data_final.drop(columns = ['Overall Rating_2022'], inplace= True)

final data -> stats and madden rating for coresponding season

There is some data on 40 time, vertical, draft position, and height (which is misinterpreted as a date in the csv file) which I will leave incase I want to use it later but so far I don't have plans to do so.

Even more madden ratings. I realized it is not that hard to find these datasets and I may as well go as far back as I can, that will give my model more data and give me access to more test data

In [254]:
#madden 20 ratings (2018 NFL season)
madden20 = pd.read_csv('Data/maddenratings/Madden20.csv')
madden20 = madden20[['Name','position','ovr rating']]
madden20 = madden20[madden20['position'] == 'WR']
madden20 = madden20[['Name','ovr rating']]

#add season
madden20['season'] = 2018
madden20['season'] = madden20['season'].astype(int)

#rename columns
madden20.rename(columns = {'Name':'name', 'ovr rating':'Overall Rating'}, inplace= True)

madden20['name'] = madden20['name'].replace('Marvin Jones Jr', 'Marvin Jones')

In [255]:
WR_data_5 = pd.merge(WR_data_final, madden20, on= ['name','season'], how = 'left',suffixes=('', '_2020'))

WR_data_5['Overall Rating'] = np.where(WR_data_5['season'] == 2018,
                                           WR_data_5['Overall Rating_2020'],
                                           WR_data_5['Overall Rating'])

WR_data_5.drop(columns = ['Overall Rating_2020'], inplace= True)

In [256]:
#madden 19 ratings for WRs (2017 NFL season)
madden19 = pd.read_excel('Data/maddenratings/Madden19.xlsx')
madden19 = madden19[['Name','Position','Overall']]
madden19 = madden19[madden19['Position'] == 'WR']
madden19 = madden19[['Name','Overall']]

#add season
madden19['season'] = 2017

#rename columns
madden19.rename(columns = {'Overall': 'Overall Rating', 'Name': 'name'}, inplace=True)

#the marvin jones issue
madden19['name'] = madden19['name'].replace('Marvin Jones Jr', 'Marvin Jones')

In [257]:
WR_data6 = pd.merge(WR_data_5, madden19, on= ['name','season'], how = 'left',suffixes=('', '_2019'))


WR_data6['Overall Rating'] = np.where(WR_data6['season'] == 2017,
                                           WR_data6['Overall Rating_2019'],
                                           WR_data6['Overall Rating'])

WR_data6.drop(columns = ['Overall Rating_2019'], inplace= True)

In [258]:
#madden 18 ratings for WRs (2016 NFL season)
madden18 = pd.read_excel('Data/maddenratings/M18&17.xlsx', sheet_name = 'M18')
madden18 = madden18[['FNAME','LNAME','POS','M18R']]
madden18 = madden18[madden18['POS'] == 'WR']
madden18 = madden18[['FNAME','LNAME','M18R']]

#add season
madden18['season'] = 2016

#create new name columm
madden18['name'] = madden18['FNAME'] + ' ' + madden18['LNAME']
madden18.drop(columns = ['LNAME','FNAME'], inplace= True)

#rename columns
madden18.rename(columns = {'M18R': 'Overall Rating'}, inplace=True)

#Marvin Jones changed his name to Marvin Jones Jr, have to fix it as he is the only active player from the
# 2012 draft class so his data has importance

madden18['name'] = madden18['name'].replace('Marvin Jones Jr', 'Marvin Jones')

In [259]:
WR_data7 = pd.merge(WR_data6, madden18, on= ['name','season'], how = 'left',suffixes=('', '_2018'))


WR_data7['Overall Rating'] = np.where(WR_data7['season'] == 2016,
                                           WR_data7['Overall Rating_2018'],
                                           WR_data7['Overall Rating'])

WR_data7.drop(columns = ['Overall Rating_2018'], inplace = True)

In [260]:
#madden 17 ratings for WRs (2015 NFL season)
madden17 = pd.read_excel('Data/maddenratings/M18&17.xlsx', sheet_name = 'M17')
madden17 = madden17[['FNAME','LNAME','POS','M17R']]
madden17 = madden17[madden17['POS'] == 'WR']
madden17 = madden17[['FNAME','LNAME','M17R']]

#add season
madden17['season'] = 2015

#create new name columm
madden17['name'] = madden17['FNAME'] + ' ' + madden17['LNAME']
madden17.drop(columns = ['LNAME','FNAME'], inplace= True)

#rename columns
madden17.rename(columns = {'M17R': 'Overall Rating'}, inplace=True)

In [261]:
WR_data8 = pd.merge(WR_data7, madden17, on= ['name','season'], how = 'left',suffixes=('', '_2017'))


WR_data8['Overall Rating'] = np.where(WR_data8['season'] == 2015,
                                           WR_data8['Overall Rating_2017'],
                                           WR_data8['Overall Rating'])

WR_data8.drop(columns = ['Overall Rating_2017'], inplace = True)

In [262]:
#madden 16 ratings (2014 NFL season)
madden16 = pd.read_csv('Data/maddenratings/Madden16.csv')
madden16 = madden16[['First Name','Last Name','Position','OVR']]
madden16 = madden16[madden16['Position'] == 'WR']
madden16 = madden16[['First Name','Last Name','OVR']]

#add season
madden16['season'] = 2014

#create new name column
madden16['name'] = madden16['First Name'] + ' ' + madden16['Last Name']
madden16.drop(columns = ['First Name','Last Name'], inplace= True)

#rename columns
madden16.rename(columns = {'OVR':'Overall Rating'}, inplace= True)

In [263]:
WR_data9 = pd.merge(WR_data8, madden16, on= ['name','season'], how = 'left',suffixes=('', '_2016'))


WR_data9['Overall Rating'] = np.where(WR_data9['season'] == 2014,
                                           WR_data9['Overall Rating_2016'],
                                           WR_data9['Overall Rating'])

WR_data9.drop(columns = ['Overall Rating_2016'], inplace = True)

In [264]:
#madden 15 ratings (2013 NFL season)
madden15 = pd.read_excel('Data/maddenratings/Madden15.xlsx', sheet_name = None)

all_teams = pd.concat(madden15.values(), keys=madden15.keys())

all_teams = all_teams[['FIRST','LAST','POSITION','OVERALL RATING']]
all_teams = all_teams[all_teams['POSITION'] == 'WR']
all_teams = all_teams[['FIRST','LAST','OVERALL RATING']]


#add season
all_teams['season'] = 2013

#create new name column
all_teams['name'] = all_teams['FIRST'] + ' ' + all_teams['LAST']
all_teams.drop(columns = ['FIRST','LAST'], inplace= True)

#rename columns, also keeping the naming consistent
all_teams.rename(columns = {'OVERALL RATING':'Overall Rating'}, inplace= True)
madden15 = all_teams

In [265]:
WR_data10 = pd.merge(WR_data9, madden15, on= ['name','season'], how = 'left',suffixes=('', '_2015'))


WR_data10['Overall Rating'] = np.where(WR_data10['season'] == 2013,
                                           WR_data10['Overall Rating_2015'],
                                           WR_data10['Overall Rating'])

WR_data10.drop(columns = ['Overall Rating_2015'], inplace = True)

In [266]:
#madden 14 ratings for WRs (2012 NFL season)
madden14 = pd.read_excel('Data/maddenratings/Madden14.xlsx')
madden14 = madden14[['First Name','Last Name','Position','Overall']]
madden14 = madden14[madden14['Position'] == 'WR']
madden14 = madden14[['First Name','Last Name','Overall']]

#add season
madden14['season'] = 2012

#create new name columm
madden14['name'] = madden14['First Name'] + ' ' + madden14['Last Name']
madden14.drop(columns = ['First Name','Last Name'], inplace= True)

#rename columns
madden14.rename(columns = {'Overall': 'Overall Rating'}, inplace=True)

In [267]:
WR_data_complete = pd.merge(WR_data10, madden14, on= ['name','season'], how = 'left',suffixes=('', '_2014'))


WR_data_complete['Overall Rating'] = np.where(WR_data_complete['season'] == 2012,
                                           WR_data_complete['Overall Rating_2014'],
                                           WR_data_complete['Overall Rating'])

WR_data_complete.drop(columns = ['Overall Rating_2014'], inplace = True)

In [268]:
WR_data_complete[WR_data_complete['name'] == 'Marvin Jones']

,id,name,position,team,season,season_type,receptions,targets,receiving_yards,receiving_tds,...,ypr,rec_td_percentage,rec_ypg,round,overall,ht,wt,forty,vertical,Overall Rating
1639,1209,Marvin Jones,WR,CIN,2012,REG,18,32,201,1,...,11.17,0.056,25.125000,5.0,166.0,2-Jun,199.0,4.46,33.0,70.0
1640,1209,Marvin Jones,WR,CIN,2013,REG,51,80,712,10,...,13.96,0.196,44.500000,5.0,166.0,2-Jun,199.0,4.46,33.0,81.0
1641,1209,Marvin Jones,WR,CIN,2015,REG,65,103,816,4,...,12.55,0.062,51.000000,5.0,166.0,2-Jun,199.0,4.46,33.0,82.0
1642,1209,Marvin Jones,WR,DET,2016,REG,55,103,930,4,...,16.91,0.073,62.000000,NaN,NaN,NaN,NaN,NaN,NaN,84.0
1643,1209,Marvin Jones,WR,DET,2017,REG,61,107,1101,9,...,18.05,0.148,68.812500,NaN,NaN,NaN,NaN,NaN,NaN,87.0
1644,1209,Marvin Jones,WR,DET,2018,REG,35,62,508,5,...,14.51,0.143,56.444444,NaN,NaN,NaN,NaN,NaN,NaN,86.0
1645,1209,Marvin Jones,WR,DET,2019,REG,62,91,779,9,...,12.56,0.145,59.923077,NaN,NaN,NaN,NaN,NaN,NaN,85.0
1646,1209,Marvin Jones,WR,DET,2020,REG,76,115,978,9,...,12.87,0.118,61.125000,NaN,NaN,NaN,NaN,NaN,NaN,83.0
1647,1209,Marvin Jones,WR,JAX,2021,REG,73,120,832,4,...,11.40,0.055,48.941176,NaN,NaN,NaN,NaN,NaN,NaN,82.0
1648,1209,Marvin Jones,WR,JAX,2022,REG,46,81,529,3,...,11.50,0.065,33.062500,NaN,NaN,NaN,NaN,NaN,NaN,78.0
